In [18]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

import pandas as pd

from itertools import combinations
from scipy import stats
from datetime import datetime, timedelta
from statsmodels.tsa.stattools import coint
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from statsmodels.tsa.stattools import adfuller
import warnings
from GetFreshMarketData import *
from tqdm import tqdm
import polars as pl
import glob
import os


trading_vol = 500000
end_date = datetime.today()
start_date = end_date - timedelta(days = 365 * 1)
MIN_DATA_POINTS = 150

In [8]:
folder_path = r"C:\Users\ksdee\Documents\Trading\Data\bhavcopy"
# Find all CSV files in the folder
list_of_files = glob.glob(os.path.join(folder_path, '*.csv'))
latest_file = max(list_of_files, key=os.path.getmtime)
index_symbols = pd.read_csv(latest_file)
index_symbols = index_symbols.loc[((index_symbols.SctySrs=='EQ') & (index_symbols.TtlTradgVol>=trading_vol)),'TckrSymb'].to_list()


INDEX_NAME = "NIFTY_500"
index_symbols_1  = pd.read_csv(fr"C:\Users\ksdee\Documents\Trading\Data\index\constituents\{INDEX_NAME}.csv")
index_symbols_1 = index_symbols_1.loc[((index_symbols_1.symbol!='DUMMYHDLVR') ),:] 
index_symbols_1 = index_symbols_1.loc[index_symbols_1.series == 'EQ','symbol'].to_list()


index_symbols = list(set(index_symbols).union(set(index_symbols_1)))


In [10]:
def trend_metrics(series, window=100):
    y = series[-window:]
    x = np.arange(len(y))
    slope, _, r, _, _ = linregress(x, y)
    return slope, r**2

def returns(series, days):
    return series.iloc[-1] / series.iloc[-days] - 1

def rsi(series, period=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

def volatility(series, window=60):
    return series.pct_change().rolling(window).std().iloc[-1]

def max_drawdown(series):
    running_max = series.cummax()
    drawdown = series / running_max - 1
    return drawdown.min()

def sharpe_like(series):
    r = series.pct_change().dropna()
    return r.mean() / r.std() if r.std() != 0 else 0

def zscore(series):
    return (series - series.mean()) / series.std()


In [26]:
records = []
for sym in tqdm(index_symbols):
    file = STOCK_DIR/f'{sym}.csv'
    if file.exists():
        df = pd.read_csv(file, low_memory=False)
        df.date = df.date.apply(lambda x : datetime.strptime(x,'%Y-%m-%d'))
        df = df.loc[df.date >= start_date,:]
        df = df.sort_values(by='date')
        
        if df.shape[0]< MIN_DATA_POINTS:
            continue

        s = df.close

        slope, r2 = trend_metrics(s)

        records.append({
                        "Ticker": sym,
                        "Slope": slope,
                        "R2": r2,
                        "Ret_1M": returns(s, 21),
                        "Ret_3M": returns(s, 63),
                        "Ret_6M": returns(s, 126),
                        "RSI": rsi(s).iloc[-1],
                        "Volatility": volatility(s),
                        "MaxDD": max_drawdown(s),
                        "SharpeLike": sharpe_like(s)
                    })

df = pd.DataFrame(records).set_index("Ticker")

positive_cols = [
    "Slope", "R2",
    "Ret_1M", "Ret_3M", "Ret_6M",
    "RSI", "SharpeLike"
]

negative_cols = ["Volatility", "MaxDD"]

for col in positive_cols:
    df[col + "_Z"] = zscore(df[col])

for col in negative_cols:
    df[col + "_Z"] = zscore(df[col])


df["MomentumScore"] = (
    0.20 * df["Slope_Z"] +
    0.15 * df["R2_Z"] +
    0.15 * df["Ret_1M_Z"] +
    0.20 * df["Ret_3M_Z"] +
    0.15 * df["Ret_6M_Z"] +
    0.10 * df["SharpeLike_Z"] -
    0.03 * df["Volatility_Z"] -
    0.02 * df["MaxDD_Z"]
)

screened = df[
    (df["Slope"] > 0) &
    (df["R2"] > 0.30) &
    (df["RSI"] > 55)
].sort_values("MomentumScore", ascending=False)

print("\nTOP MOMENTUM STOCKS\n")
print(
    screened[
        ["MomentumScore", "Ret_1M", "Ret_3M", "Ret_6M", "RSI"]
    ].round(3)
)

screened.to_csv(TEMP/"momentum_screener_output.csv")

100%|██████████| 833/833 [00:32<00:00, 25.39it/s]


TOP MOMENTUM STOCKS

            MomentumScore  Ret_1M  Ret_3M  Ret_6M     RSI
Ticker                                                   
SILVERIETF          2.421   0.408   0.727   1.509  81.311
SILVER1             2.421   0.407   0.736   1.508  82.365
SILVERADD           2.394   0.404   0.718   1.508  80.877
SILVERAG            2.378   0.399   0.721   1.492  80.306
SILVER              2.376   0.408   0.713   1.500  77.343
...                   ...     ...     ...     ...     ...
YESBANK             0.205   0.087   0.013   0.158  71.754
MON100              0.177   0.006  -0.006   0.224  56.917
CHENNPETRO          0.162  -0.062   0.166   0.145  55.384
MARICO              0.123   0.028   0.046   0.037  56.864
TORNTPOWER          0.041   0.068   0.017  -0.024  59.828

[116 rows x 5 columns]


,Slope,R2,Ret_1M,Ret_3M,Ret_6M,RSI,Volatility,MaxDD,SharpeLike,Slope_Z,R2_Z,Ret_1M_Z,Ret_3M_Z,Ret_6M_Z,RSI_Z,SharpeLike_Z,Volatility_Z,MaxDD_Z,MomentumScore
Ticker,,,,,,,,,,,,,,,,,,,
SILVERIETF,1.239011,0.824134,0.407663,0.727245,1.509350,81.310884,0.023587,-0.198425,0.258598,0.242946,0.982268,3.424447,3.966491,4.715958,2.137728,2.405947,0.510766,0.705140,2.421457
SILVER1,1.209224,0.825467,0.407069,0.735961,1.508471,82.364873,0.024197,-0.198292,0.251938,0.238992,0.986543,3.419324,4.012082,4.713254,2.199249,2.339947,0.580567,0.706132,2.420538
SILVERADD,1.198151,0.824188,0.403871,0.717822,1.507606,80.877256,0.024330,-0.207977,0.249053,0.237523,0.982440,3.391743,3.917208,4.710589,2.112417,2.311354,0.595685,0.634092,2.394245
SILVERAG,1.201227,0.822572,0.399222,0.721327,1.491799,80.306477,0.024268,-0.223872,0.240213,0.237931,0.977256,3.351649,3.935538,4.661930,2.079101,2.223748,0.588601,0.515854,2.377719
SILVER,1.231718,0.814532,0.408355,0.712763,1.500445,77.343343,0.025330,-0.248894,0.234109,0.241978,0.951468,3.430421,3.890744,4.688545,1.906144,2.163254,0.710009,0.329731,2.375540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YESBANK,0.022552,0.342789,0.086735,0.013408,0.157609,71.753986,0.013767,-0.203608,0.055711,0.081495,-0.561780,0.656498,0.232874,0.554866,1.579896,0.395298,-0.611446,0.666588,0.204853
MON100,0.303314,0.444066,0.006148,-0.006073,0.224185,56.916996,0.008269,-0.260247,0.038502,0.118758,-0.236906,-0.038552,0.130980,0.759808,0.713868,0.224753,-1.239894,0.245276,0.177367
CHENNPETRO,2.222999,0.329351,-0.061546,0.166066,0.144898,55.384333,0.036274,-0.286216,0.063801,0.373543,-0.604888,-0.622398,1.031328,0.515737,0.624407,0.475468,1.960770,0.052107,0.161923
